# Processing Data Using Azure Synapse Analytics

This lab will introduce you to Synapse workspaces and cover the following recipes:

* Provisioning an Azure Synapse Analytics workspace
* Analyzing data using serverless SQL pool
* Provisioning and configuring Spark pools
* Processing data using Spark pools and a lake database
* Querying the data in a lake database from serverless SQL pool
* Scheduling notebooks to process data incrementally
* Visualizing data using Power BI by connecting to serverless SQL pool

By the end of the lab, you will have learned how to provision a Synapse workspace and Spark pools, explore and analyze data using serverless SQL pool and Spark pools, create a lake database and query a lake database from serverless SQL pool and a Spark pool, and finally, visualize the lake database data in Power BI.

## Recipe 1 - Provisioning an Azure Synapse Analytics workspace

In this recipe, we’ll learn how to provision a Synapse Analytics workspace. A Synapse Analytics workspace is the logical container that will hold the Spark pools, SQL pool, and integration pipelines that are required for the data engineering tasks.

Follow these steps to create a Synapse Analytics workspace:

1. Go to the Azure portal home page at portal.azure.com and click on Create a resource. Search for Synapse and select Azure Synapse Analytics. Click the Create button.
1. Provide a unique workspace name. For this example, we are using sparshadesynapse. You may pick any location.
1. Provisioning a Synapse Analytics workspace requires an Azure Data Lake Storage Gen2 account. You may use an existing account or create a new account using the Create new link.
1. A Synapse Analytics workspace requires a container on the new data lake account provisioned. Let’s create a new container named `synapse` by clicking the Create new link. After the Synapse workspace is provisioned, the Synapse workspace service account is given permissions on the data lake account and the container by default. We will be able to connect to the data stored in the storage account from the Synapse workspace seamlessly. The data lake account will be used to store a lake database (a component of Synapse Spark pools) and other artifacts of the Synapse workspace. Click the Next: Security > button at the bottom.
1. On occasion, Azure subscriptions may not be able to create a Synapse workspace because of a Resource provider not registered for the subscription error. To resolve the error, go to the Azure portal and open your subscription. Click the Resource providers section, search for the Synapse resource, and register Microsoft.Synapse on the subscription. Please refer to https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/resource-providers-and-types#azure-portal for detailed steps on how to register a resource on a subscription.
1. On the Security page, we will be filling in the user ID and password details for SQL pool. Provide the user ID and password as sqladminuser and sparshAdeSynapse123. Click Review + Create to create the Synapse Analytics workspace.
1. The preceding steps will create a Synapse Analytic workspace and a storage account.

![](https://user-images.githubusercontent.com/62965911/218262814-16ccdb7f-5287-4e08-bacd-e7d7117eae23.png)

![](https://user-images.githubusercontent.com/62965911/218262911-e30adcf8-3499-4e91-b771-0dcbffd455f6.png)

## Recipe 2 - Analyzing data using serverless SQL pool

Serverless SQL pool allows us to explore data using T-SQL commands in a Synapse Analytics workspace. The key advantage of serverless SQL pool is that it is available by default once a Synapse Analytics workspace is provisioned with no cluster or additional resources to be created. In serverless SQL pool, you will be charged only for the data processed by the queries as it is designed as a pure pay-per-use model.

In this recipe, we will perform the following:

1. Log in to portal.azure.com, go to All resources, and search for sparshadesynapse, the Synapse Analytics workspace we created in the Provisioning an Azure Synapse Analytics workspace recipe. Click the workspace. Click Open Synapse Studio.
1. Click on the blue cylinder (the data symbol) on the left, which will take you to the Data section. Click the Linked tab. Expand the data lake account of the Synapse workspace (sparshadesynapse for this example) and click on the synapse (Primary) container. Click the + New folder button and create a folder called CSV.
1. Double click on the CSV folder in Synapse Studio. Click on the Upload button and upload the covid-data.csv file from your local machine into the data lake account of the Synapse workspace.
1. After the file has uploaded, right-click the covid-data.csv file in the CSV folder, select New SQL script, and click the Select TOP 100 rows option.
1. A new query window will open with a script using the OPENROWSET command. Click the Run button to execute the query and preview the data.
1. Let’s perform the following. Let’s create a serverless database and a view referencing the OPENROWSET command to read the covid-data.csv file. We notice that the actual column names (iso_code, continent, location, and date) are listed in the first row. These columns (iso_code, continent, location, and date) need to move up to become the table’s column names and we need to remove the existing column names (C1, C2, C3, C4, and so on). We can fix that by adding the HEADER_ROW = TRUE option after the PARSER_VERSION option in the OPENROWSET command. Use the following command to create a database, create a view, and fix the header:
    ```sql
    CREATE DATABASE serverless

    GO

    USE serverless

    GO

    CREATE VIEW covid AS
    SELECT
    *
    FROM
        OPENROWSET(
            BULK 'https://sparshstorage1.dfs.core.windows.net/synapse/CSV/covid-data.csv',
            FORMAT = 'CSV',
    PARSER_VERSION = '2.0'
    , HEADER_ROW = TRUE
        ) AS [result]
    ```
1. Click on the blue icon that looks like a notebook on the left-hand side of the screen. This will take you to the Develop section.
1. Click the + button on top and click SQL script.
1. Use the following script, referencing the serverless database and the view created, to find a list of countries that have the maximum number of deaths per million people on a given day:
    ```sql
    use serverless

    GO

    Select iso_code,location,continent,
    max(isnull(new_deaths_per_million,0)) as death_sum,
    max(isnull(people_fully_vaccinated,0) / isnull(population,0)) * 100 as percentage_vaccinated From covid
    where isnull(population,0) > 1000000
    group by iso_code,location,continent
    order by death_sum desc
    ```

The output of the preceding query is shown in the following screenshot:

![](https://user-images.githubusercontent.com/62965911/218263794-d12364d7-c432-484f-a998-1e1e9aeac720.png)

How it works…

After we uploaded the covid-data.csv file to the Azure Data Lake Storage account associated with the Synapse workspace, we were able to query the data at the click of a button, without providing any credentials or provisioning any other resources. Serverless SQL pool, which is available by default in a Synapse workspace, allows us to interact with the data with minimal effort.

We used the OPENROWSET function to read the data from a CSV file and we encapsulated it inside a view for easier access in subsequent scripts. The serverless database and the view can also be accessed from other services such as Power BI and Data Factory. Serverless SQL pool, with its ability to define views, can be used to create a logical data warehouse on top of a data lake storage account, which will serve as a powerful tool for data analysis and exploration.

## Recipe 3 - Provisioning and configuring Spark pools

A Spark pool is an important component of Azure Synapse Analytics that allows us to perform data exploration and processing using the Apache Spark engine. Spark pools in Azure Synapse Analytics allow us to process data using programming languages such as PySpark, Scala, C#, and Spark SQL. In this recipe, we will learn how to provision and configure Spark pools in Synapse Analytics.

Let’s perform the following steps to provision a Spark pool in an Azure Synapse Analytics workspace:

1. Log in to portal.azure.com and click All Resources. Search for sparshadesynapse, the Synapse Analytics workspace created in the Provisioning an Azure Synapse Analytics workspace recipe. Click on the workspace. Search for Apache Spark pools under Analytics pools. Click + New.
1. Name the Spark pool sparshsparkpool.
1. The Node size family property indicates the type of virtual machines that will be running to process the big data workload. Let’s pick Memory Optimized, as it’s typically good enough for general purpose data processing tasks. The Hardware accelerated type provides GPU-powered machines meant for performing heavy-duty data science and big data workloads.
1. The Node size property indicates the compute and memory of virtual machines in a Spark pool. Let’s pick Small (4 vCores / 32 GB), as it’s the cheapest option.
1. Autoscale allows the Spark pool to allocate additional nodes or machines depending upon the workload. Let’s leave it as Enabled.
1. Set the minimum and maximum Number of nodes to 3 and 10. A Spark pool can autoscale up to a maximum of 10 machines/nodes. Click on Next: Additional settings >.
1. Leave Automatic pausing as Enabled. Automatic pausing stops the Spark pool when there are no jobs being processed.
1. Set Number of minutes idle to 10. This ensures that the pool is paused if no job is running for 10 minutes.
1. Set the Apache Spark version to the latest one (3.2 here), as it ensures that we get the latest Java, Scala, .NET, and Delta Lake versions. Click Review + create.

How it works…

Creating a Spark pool defines the configuration for the nodes/virtual machines, which will be processing the big data workload as it arrives. Each time a new user logs in and submits a Spark job/Spark notebook to process data, an instance of Spark pool is created. An instance of Spark pool is basically a bunch of virtual machines/nodes configured as defined in the Spark pool configuration. A single user can use up to the maximum number of nodes defined in the pool. If there are multiple users connecting to the Spark pool, Synapse will create as many instances of Spark pool as the number of users. You will be billed for the number of active instances and for the time period during which they were active. Billing will stop for a particular instance if it remains idle longer than the idle time period defined in the Spark pool configuration.

## Recipe 4 - Processing data using Spark pools and a lake database

Spark pools in a Synapse workspace allow us to process data and store them as tables inside a lake database. A lake database allows us to create tables using CSV files, Parquet files, or as Delta tables stored in the data lake account. Delta tables use Parquet files for storage and support insert, update, delete, and merge operations. Delta tables are stored in a columnar format, which is compressed, ideal for storing processed data and supporting analytic workloads. In this recipe, we will read a CSV file, perform basic processing, and load the data into a Delta table in a lake database.

Let’s perform the following steps to process the data in the CSV file and load it into a Delta Lake table in a lake database:

1. Log in to portal.azure.com, click All resources, search for sparshadesynapse, the Synapse Analytics workspace we created, and click on it. Click Open Synapse Studio.
1. Click on the data icon on the left, click the Linked tab, and expand the Azure Data Lake Storage Gen2 | sparshadesynapse | synapse (Primary) container. Navigate to the CSV folder inside the synapse container, where the covid-data.csv file has been uploaded. Right-click on the covid-data.csv file, select the option New notebook | Load to DataFrame.
1. A new notebook will be created. Notebooks are used by data engineers to develop the code that will be used to process data using Synapse pools. Attach the notebook to sparshsparkpool, the Synapse Spark pool created in the Provisioning and configuring Spark pools recipe in this lab, using the Attach to drop-down menu at the top of the notebook. Uncomment the fourth line in the first cell by removing ##, as our file contains a header. Hit Run Cell (which looks like a play button) on the left.
1. Data will be loaded to a DataFrame called df using the automatic PySpark code generated. Let’s use the Spark SQL language to understand the data. To interact with the data using Spark SQL, we need to create a temporary view. The createOrReplaceTempView command helps to create a temporary view that will be visible only within the notebook. Add a new cell by hitting the + Code button, paste the following command, and run the new cell:
    ```py
    df.createOrReplaceTempView("v1")
    ```
1. To check out which columns are present in the covid-data.csv file, let’s use the Describe Spark SQL command to list the columns in the view. Add a new cell and paste the following command. The %%sql command switches the programming language from PySpark to Spark SQL. We will notice that the view contains several columns (use the scroll bar to check out all of them). All the columns are also of the string data type:
    ```py
    %%sql
    Describe v1;
    ```
1. Let’s focus on the following key columns – date, continent, location, new_cases, and new_deaths. Let’s also change the data type of new_cases and new_deaths to integer and load it into a Delta table. To load it into a Delta table, we need to create a lake database first, create the new Delta table, and then load the data. The Create database command creates the database, and Create table <tablename> using Delta as <Select statement> creates the table and loads the data. Copy the following command to a new cell and run the new cell:
    ```py
    %%sql
    Create database sparksqldb;
    Create or replace table sparksqldb.covid
    USING Delta
    AS
    Select date, continent,location, CAST(new_cases as int) as new_cases,
    CAST(new_deaths as int) as new_deaths from v1
    ```

1. In the previous step, we created a lake database called sparksqldb and a Delta table inside it called covid. Using the delta option in the CREATE or REPLACE TABLE command ensured that the table was created as a Delta table. The CAST function in the SELECT statement changed the column data type to INTEGER. Verify the data-type change using the DESCRIBE command:
    ```py
    %%sql
    Describe table sparksqldb.covid;
    ```
1. Let’s delete the rows that have NULL values in the continent column. Add a new cell and copy-paste the following command:
    ```py
    %%sql
    Delete from sparksqldb.covid where continent is NULL
    ```
1. Delta tables have a feature called time travel, which lets us explore the previous versions of the table. We will use time travel to query the deleted rows (rows with NULL values in the continent column). To perform that as a first step, we need to find the location where the Delta table is stored. The Describe detail command will provide a column called location, which will contain the location of the Delta table. Add a new cell, copy the following command, and run the cell. Copy the contents of the location column. Ensure to expand the column by dragging the slider to your right and copying the full path of the Delta table:
    ```py
    %%sql
    DESCRIBE DETAIL sparksqldb.covid  
    ```
1. On the copied location, remove the text that starts with abfss and goes up to windows.net. We only need the path that starts from the container name (synapse), not the storage account or protocol details. For example, if your copied location is abfss://synapse@sparshadesynapse.dfs.core.windows.net/synapse/workspaces/sparshadesynapse/warehouse/sparksqldb.db/covid, remove abfss://synapse@sparshadesynapse.dfs.core.windows.net/ and retain /synapse/workspaces/sparshadesynapse/warehouse/sparksqldb.db/covid.
1. Add a new cell and copy the following Spark command. Paste the edited location path to the load function. The Option("versionAsOf",0) function makes the command read the older version of the table. The second parameter in the option function indicates the version number to be read. Version number 0 would be the most recent previous version of the table, version number 1 would be the next version older than version 0, and so on. The command reads the older data to a DataFrame, which we load to a view called old_Data:
    ```py
    df2 =  spark.read.format("delta").option("versionAsOf", 0).load("/synapse/workspaces/sparshadesynapse/warehouse/sparksqldb.db/covid")
    df2.createOrReplaceTempView("old_Data")
    ```
1. Execute a SELECT statement against the old_Data view to check out the rows that were deleted. Add a new cell, copy the following command, and execute the new cell. We will be able to read the deleted rows using the time travel feature on the Delta table:
    ```py
    %%sql
    SELECT * FROM old_Data WHERE continent IS NULL
    ```

How it works…

The Spark pools in a Synapse workspace allow us to seamlessly load CSV files to Delta tables using notebooks. Notebooks allow us to effortlessly switch between PySpark and SQL. Delta tables support data manipulation commands such as update, delete, and merge, and capabilities such as time travel make it very efficient for data processing tasks in data engineering projects.

NOTE: Related assets are in assets folder.

## Recipe 5 - Querying the data in a lake database from serverless SQL pool

Lake databases are created from Synapse Spark pools and typically consist of Delta tables. The following recipe will showcase how we could read the data stored in Delta tables from serverless SQL pool.

Perform the following steps to query the data:

1. Log in to portal.azure.com, click All Resources, search for sparshadesynapse, the Synapse Analytics workspace that we created, and click on it. Click Open Synapse Studio. Click on the data icon on the left, click the Linked tab, and expand the Azure Data Lake Storage Gen2 | sparshadesynapse | synapse (Primary) container. Navigate to the following folder path: /synapse/workspaces/sparshadesynapse/warehouse/sparksqldb.db. The usual path structure is <SynapseContainerName>/synapse/workspaces/<WorkspaceName>/warehouse/<lakedatabasename.db>. So, if you have named your lake database or table name differently, then it will vary from mine here. You will find a folder with the Delta table name (covid). Right-click on it and select New SQL script | Create external table.
1. Synapse will detect the file type and schema. Hit Continue to generate the external table creation script.
1. Leave the Select SQL pool option as Built-in. Built-in is the in-built serverless SQL pool. Select + New to create a new serverless database.
1. Name the database ServerlessSQLdb and click the Create button.
1. Name the external table `[dbo].[covid_ext]`. Click Open script.
1. To create an external table in Synapse SQL pool, we need to create the following objects: an external file format and an external data source first. Using Synapse Studio, we can generate the script to create the external file format, external data source, and external table. Select ServerlessSQLdb from the Use database dropdown and click the Run button to create the external table.
1. Upon clicking the Run button, we will be able to see the data read from the external table successfully.

![](https://user-images.githubusercontent.com/62965911/218266160-3b073bfc-f4be-411f-af5a-114a3ad67d84.png)

How it works…

To access the Delta table in a lake database, we need to create an external table in serverless SQL pool. We identified the folder where the Delta table was stored and we created an external table against it in serverless SQL pool. The external table acts as a link to the files stored in the Delta table. While the files reside in the Delta table, files appear as a table to end users in Serverless SQL pool. So, when a user queries the external table using a T-SQL script in serverless SQL pool, it will seamlessly read from the Delta table’s files and present it in a tabular format.

The lake database created a folder for each Delta table. So, it made it easier for us to create an external table against the folder of the Delta Lake table, which implies that we can seamlessly query the data from lake database Delta table in a serverless SQL pool database via external table. Changes to the Delta table are handled by adding or removing Parquet files inside the Delta Lake table folder using the Apache Spark engine. As we have created the external table against the table’s folder (not against any specific file), all the changes happening in the Delta Lake table will immediately be reflected in the serverless SQL pool’s external table without any additional effort.

## Recipe 6 - Scheduling notebooks to process data incrementally

Consider the following scenario. Data is loaded daily into the data lake in the form of CSV files. The task is to create a scheduled batch job that processes the files loaded daily, performs basic checks, and loads the data into the Delta table in the lake database. This recipe addresses this scenario by covering the following tasks:

- Only reading the new CSV files that are loaded to the data lake daily using Spark pools and notebooks
- Processing and performing upserts (update if the row exists, insert if it doesn’t), and loading data into the Delta lake table using notebooks
- Scheduling the notebook to operationalize the solution

1. Create a folder called transaction-data inside the synapse container in the sparshadesynapse data lake account. You can use Synapse Studio’s data pane to do the same. Upload the `TransDtls-2023-02-11.csv` file in this folder.
1. In this scenario, the TransDtls-2023-02-11.csv file contains the data about transactions that have occurred in a store. Let’s assume that the file has the loading date suffixed to it. So, a file that was loaded to the data lake on Feb 11th will be named TransDtls-2023-02-11.csv, a load from Feb 12th will be named TransDtls-2023-02-12.csv, and so on. Our notebook and scheduled task should read only the latest file (and not all the files in the transaction-data folder), so that it can process the data incrementally.

To process the data incrementally and load the data to a Delta Lake, let’s perform the following steps:

1. Click the Develop icon on the left-hand side of Synapse Studio. In the Notebooks section, click on the three dots and select New notebook.
1. Name the notebook Incremental_Data_Load by typing the name into the Properties section on the left. Attach the notebook to the sparshsparkpool cluster using the Attach to drop-down option at the top. Copy the following Scala script and paste it into the first cell in the notebook to only read the latest file from the transaction-data folder into a DataFrame. The Java.time.localDate.now command gets the current date and we use the current date to construct the name of the file to be read. This way, even if there are hundreds of files in the folder, the notebook will only read the latest file. Hit the Run button (which looks like a play button) on the left. The latest file is loaded to the DataFrame and named transaction_today:
    ```py
    %%spark
    val date = java.time.LocalDate.now
    val transaction_today = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/transaction-data/TransDtls-" + date +".csv")
    display(transaction_today)  
    ```
1. Add a new cell using the + Code button and copy-paste the command to create a temporary view using the DataFrame. Hit the Run button:
    ```py
    %%spark
    transaction_today.createOrReplaceTempView("transaction_today")
    ```
1. Add a new cell and use the following SQL script to create a lake database called Dataload and a Delta table called transaction_Data. The If not exists clause in the create statements ensures that the database and the table are only created the first time that the notebook is run. Hit the Run button to create the database and table:
    ```py
    %%sql
    CREATE DATABASE IF NOT EXISTS DataLoad;
    CREATE TABLE IF NOT EXISTS DataLoad.transaction_data(transaction_id int, order_id int, Order_dt Date,customer_id varchar(100),product_id varchar(100),quantity int,cost int)
    USING DELTA
    ```
1. Add a new cell and copy the following script to upsert data into the table. If the latest file in the transaction-data folder contains information about transactions that already exist in the Delta table, then the Delta table needs to be updated with the values from the latest file, and if the latest file contains transactions that don’t exist in the Delta table, then they need to be inserted. The script uses the merge command, which performs the following tasks to achieve the update/insert commands:
    - Compares transaction_id on the latest file in the transaction-data folder (using the transaction_today view’s transaction_id column) with the transaction_data table’s (as in, the Delta table’s) transaction_id column to see whether the file contains data about older transactions or new transactions. The comparison is carried out using the merge statement’s ON clause.
    - If transaction_id from the file already exists in the transaction_data table, it implies that the file contains rows about older transactions, and hence, it updates all the columns of the transaction_data table with the latest data from the file. The WHEN Matched clause in the merge statement helps to achieve this.
    - If transaction_id doesn’t exist in the Delta table but it does exist in the file, it is a new transaction and it is therefore inserted into the table. The WHEN NOT Matched clause in the merge statement helps to achieve this.
    - On the WHEN Matched clause, additional NULL checks are added, using the is not null clause to ensure that invalid rows are not inserted into the table:
    ```py
    %%sql
    Merge into DataLoad.transaction_data source
    Using transaction_today target on source.transaction_id = target.transaction_id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED AND (target.transaction_id is not null or target.order_id is not null or target.customer_id is not null)
    THEN INSERT *
    ```
1. Hit the Publish button at the top to save the notebook.
1. To schedule the notebook to run daily, we need to add the notebook to a pipeline. Hit the add to pipeline button in the top-right corner and select New pipeline.
1. Name the pipeline Incremental_Data_Load. Publish it by clicking the Publish button. Click Add trigger and select New/Edit.
1. Click + New under Add triggers.
1. Name the trigger Incremental_Data_Load. Under Recurrence, set the schedule to run every 1 day. Under Execute at these times, type in 9 for Hours and 0 for Minutes. Click OK to schedule.
1. There are no parameters to be passed. Hit OK and proceed.
1. Hit the Publish all button to finish scheduling the notebook via a pipeline for a daily run.

![](https://user-images.githubusercontent.com/62965911/218267595-32be3855-0a9b-43d1-9022-4979dca93b7d.png)

How it works…

Processing data incrementally is a common scenario within data engineering projects. Synapse notebooks are extremely powerful and can be used to identify the new files to be loaded, process them alone, and load them into a Delta table. The MERGE statement is very effective at identifying the new or old transaction records and performing insert/update on the Delta table accordingly. The notebook was added to a pipeline at the click of a button and the pipeline was scheduled to run daily to process the files that are loaded every day. The processed Delta table, which is the outcome of this recipe, is typically consumed by a reporting application such as Power BI to get insights out of the processed data.